# Laboratorio 8. EAE253b
<hr/>

**Antes de comenzar:**

- Laboratorio debe ser realizado **de forma individual**. Obviamente, se pueden discutir ideas, pero cualquier intercambio de códigos **no está permitido**.
- Sólo se evaluarán secciones "**TAREAS**"; pero se recomienda realizar secciones **EJERCICIOS**

**Instrucciones de entrega:**

- Debe entregar este laboratorio por WebCurso, en buzón de tareas. Descargar archivo ".ipynb" a su equipo y luego subirlo.
- Además, **debe compartir el Notebook con ayudantes (arybertt@uc.cl, pagonzalez20@uc.cl) y profesor (cealvara@gmail.com)**.
- Plazo máximo de entrega: **Viernes 14 de junio, 11.55 pm.**

**Tutorial:**

- Si quieren ver más ejemplos de uso de K-means, pueden mirar este pequeño tutorial:

http://www.aprendemachinelearning.com/k-means-en-python-paso-a-paso/



<hr />

## Introducción

Con lo aprendido hasta ahora podemos importar una base de datos y realizar cierto nivel de análisis con estos. Además, tras el curso de econometría, hemos aprendido sobre las regresiones lineales... 

En este laboratorio practicaremos algunas técnicas básicas de machine learning, como lo son los métodos de agrupación o **clustering**. Estos métodos forman parte de las técnicas de machine learning denominadas **"no supervisadas"**, ya que el problema a resolver no involucra alguna variable de interés ("Y", o variable dependiente), sino que sólo involucra mucha información respecto alguna unidad observacional (o sea, tenemos muchas "X"). El problema, entonces, es resumir la información de alguna manera para poder interpretar o representar (gráficar, visualizar) de manera más simple los datos.

En concreto, en este lab veremos el algoritmo "K-means" o "K-medias", que es un método que, sin necesitar de una variable dependiente, nos permite agrupar nuestras observaciones de forma **discreta** en base a la "distancia" entre ellas (recordemos que existen otros métodos que nos permiten agrupar nuestros datos en forma **continua**, por ejemplo, PCA).


*Nota: Ejemplo basado en workshop "Machine Learning con Python", realizado por DataUC*

### Importamos algunas herramientas necesarias

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

A continuación, tal como hacemos usualmente, cargamos los datos del lab

In [ ]:
link = 'https://drive.google.com/file/d/11W1Rkz3_4j0YRPX34AZ-nTi6ZFWmTRi-/view?usp=sharing'
id = link.split('/')[5]
print(id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ejemplo.sqlite')

In [ ]:
conn = sqlite3.connect('ejemplo.sqlite')
df = pd.read_sql('SELECT * FROM data where A is not null and b is not null and class is not null', conn)

#veamos algunas estadísticas de esta base de datos
df.describe().transpose()

La base de datos que acabamos de cargar ("df") tiene 3 columnas (A, B y Class).

A y B corresponden a variables dependientes. Por razones pedagógicas, también esta base de datos incluye una variable que podría ser de interés ("y"). Ocuparemos esta variable para poder contrastar y entender mejor lo que hace el algoritmo K-medias.

Veamos gráficamente qué son las variables "A" y "B":

In [ ]:
plt.scatter(df['A'], df['B'],alpha=0.5)

## Métodos de agrupación: K-medias

K-medias es un método de agrupación, que tiene como objetivo la partición de un conjunto de n observaciones en k grupos en el que cada observación pertenece al grupo cuyo valor medio es más cercano. Con este se busca formar grupos que sean heterogéneos entre grupos, pero homogéneos dentro de cada uno.

Este método tiene diversas aplicaciones, una de las cuales consiste en la segmentación de mercado. En términos matemáticos, la lógica es la siguiente:

<center>
  $ arg\:\min\: \sum_{i=1}^{k}  \sum_{x_j\in S_i} ||x_j-\mu_i ||^2  $  

  Donde $k$ es el número de clusters, $S_i$ es el cluster $i$ y $\mu_i$ corresponde a la media de dicho cluster.
</center>

 <br/>
 
**Para realizar un análisis de conglomerados, primero debemos determinar la cantidad de grupos que queremos formar.** 

El siguiente código nos permite analizar, para diversos números de grupos, la "inercia" resultante dado un número de grupos particular. La inercia es básicamente una medida de qué tan distintos son los miembros de los grupos resultantes entre si. Si eligieramos un número de grupos igual al número de observaciones, entonces lograríamos inercia=0, pero lo que buscamos es crear el menor número de grupos posibles, ya que precisamente queremos poder agrupar la información en pocos grupos.

In [ ]:
#el objeto "StandardScaler" nos permite escalar las variables para evitar problemas con distancias absolutas..

from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
Xs = scl.fit_transform(X) #Estandarizamos variables


from sklearn.cluster import MiniBatchKMeans
dist = []
for i in range(1, 11):
    km = MiniBatchKMeans(n_clusters=i, init = 'k-means++', \
                n_init = 10, max_iter = 300,random_state = 1, batch_size = 1000)
    _ = km.fit(Xs)
    dist.append(km.inertia_)
    
plt.plot(range(1, 11), dist, 'o-');
plt.ylabel('Inercia');
plt.xlabel('Numero de Clusters');


En esta ocasión, determinamos el número optimo de clusters buscando el punto de inflexión de la curva. En nuestro caso el numero optimo se encuentra entre 3 y 4 clusters.


**A continuación entrenamos nuestro modeo utilizando 3 clusters y luego graficamos los grupos proyectados a 2 dimensiones especificas.**

Apliquemos entonces el método "Kmeans" con nuestros datos:


In [ ]:
from sklearn.cluster import KMeans

clusters = 3

cluster = KMeans(n_clusters = clusters, random_state = 0, init='k-means++')

# Notar que aquí usamos Xs, que son los datos originales, 
# pero normalizados (recuerden el problema de la "distancia" que mencioné en clases)

l  = cluster.fit_predict(Xs)

# la función "fit_predict" entrega la clasificación (grupo) predicha para cada observación. 
# Guardamos esa información en "l", para poder graficar posteriormente

In [ ]:
x, y = X[:, 0], X[:, 1]
cc = scl.inverse_transform(cluster.cluster_centers_)

for i in range(clusters):
    _ = plt.scatter(x[l == i], y[l == i], s = 1)
    _ = plt.scatter(cc[i, 0], cc[i, 1], c = 'C{}'.format(i), s = 100)

**Como podemos ver en los resultados, en comparación al gráfico realizado previamente, ahora sí es posible distinguir las diferencias que existen entre las clases.**

## Tarea 1: Preguntas teóricas (5 puntos)

1. ¿Qué diferencias hay entre un modelo supervisado y uno no supervisado? ¿A cuál de estos correspondería el análisis de conglomerados?
2. ¿Qué paso es necesario realizar con los datos previo a utilizar un modelo no supervisado? ¿Por qué? 
3. ¿Por qué cree es necesario aplicar criterios al momento de decidir la cantidad de clusters? ¿Qué ocurriría si elige muchos o muy pocos? Busque otro criterio de decisión y argumente su uso sobre el criterio antes presentado.
4. ¿Por qué cree que métodos como éste pueden ser útiles en su carrera?
 

**Ingrese sus respuestas aquí:**

R1:

R2:

R3:

R4:

## Tarea 2: Aplicación (5 puntos)

A continuación usted deberá aplicar sus conocimientos de clustering. Para ello importaremos el set de datos "Iris", un set que se ha confertido en un caso de prueba común para por muchas técnicas de clasificacion estadísticas en machine learning.

Edgar Anderson coleccionó estos datos para cuantificar la variación morfológica de la flor Iris de tres especies relacionadas (Iris setosa, Iris virginica e Iris Versicolor).

El conjunto de datos contiene 50 muestras de cada una de tres especies de Iris, para las que se midieron el largo y ancho del sépalo y del pétalo, todos en centímetros. Basado en la combinación de estos cuatro rasgos, Fisher desarrolló un modelo discriminante lineal para distinguir entre una especie y otra.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
Y = iris.target

datos = pd.DataFrame(X,columns = ['Sepal Length','Sepal Width','Petal Length','Petal Width'])
datos.head()

En base a la base de datos cargada, realizando un análisis de conglomerado, usted deberá: 

1. Evaluar la importancia de escalar las variables y realizarlo en caso de determinarlo necesario.
2. Determinar el número óptimo de clusters a utilizar.
3. Graficar sus resultados, mostrando como se puede de clasificar a estas plantas en base a sus características.
4. Explicar qué ventaja nos puede presentar utilizar este método en vez de otros métodos tales como una regresión.

In [ ]:
# Realice las operaciones aquí


In [ ]:
# Realice las operaciones aquí

<hr/>

<hr/>
<center> <h1>Fin del laboratorio.</h1> </center>

 **Recuerde compartir el Notebook con ayudantes (arybertt@uc.cl, pagonzalez20@uc.cl) y profesor (cealvara@gmail.com)**.